In [1]:
# basic functions
from nipype.interfaces.ants import ANTS
import os,time
OUT_DIR='/output/PD_ICA/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_09_masked='/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz'

# read group subject images given template
def dataGraber_sub(SUB_ID, DATA_DIR, TMPT_STR):
    import nipype.interfaces.io as nio
    import time
    t0=time.time()
    print('Grabbing files for: ', SUB_ID)
    OUT_FILE=[]
    out_len=len(TMPT_STR)
    if out_len == 0:
        print(SUB_ID+' has no files named: ', TMPT_STR)
        return OUT_FILE
    else:
        for i in range(out_len):
            TMP='%s/anat/%s_'+TMPT_STR[i]
            ds = nio.DataGrabber(infields=['subject_id', 'subject_id'])
            ds.inputs.base_directory = DATA_DIR # database
            ds.inputs.template = TMP 
            ds.inputs.subject_id = [SUB_ID]
            ds.inputs.sort_filelist = True
            res = ds.run()
            res_list = res.outputs.outfiles
            OUT_FILE.append(res_list)
        #print(SUB_ID+' files: ', OUT_FILE)
        print('dataGraber takes: ', time.time()-t0 )
        return OUT_FILE
    
# read subject affince and deformation fields from fMRIPrep results
def h5toWarp_nii(H5_FILE, OUT_DIR):
    from nipype.interfaces.ants import CompositeTransformUtil
    import os
    import time
    t0=time.time()
    tran = CompositeTransformUtil()
    tran.inputs.process = 'disassemble'
    tran.inputs.in_file = H5_FILE
    tran.inputs.out_file = OUT_DIR #bug
    #print(tran.cmdline)
    res=tran.run()
    out_warp  =OUT_DIR+'_Warp.nii.gz'
    out_affine=OUT_DIR+'_Affine.txt'
    os.system('mv '+res.outputs.displacement_field+' '+out_warp )
    os.system('mv '+res.outputs.affine_transform+' '+ out_affine)
    print('.h5 file disassemble takes: ', time.time()-t0 )
    return [out_warp, out_affine]

# Get masked subject Jacobians
def GetJacobian_nii(IN_IMAGE, MASK, JACOB_IMG, JACOB_MASK_IMG):
    # Jacobian of deformation field
    from nipype.interfaces.ants import CreateJacobianDeterminantImage
    from nipype.interfaces import fsl
    import time
    t0=time.time()
    jacobian = CreateJacobianDeterminantImage()
    jacobian.inputs.imageDimension = 3
    jacobian.inputs.deformationField = IN_IMAGE
    jacobian.inputs.outputImage = JACOB_IMG
    jacobian.inputs.num_threads = 4
    #print(jacobian.cmdline)
    jacobian.run()
    mask = fsl.ApplyMask(
        in_file=JACOB_IMG,
        out_file=JACOB_MASK_IMG,
        mask_file=MASK)
    mask.run()
    print('Jacobian takes: ', time.time()-t0 )
    return JACOB_MASK_IMG

201003-18:13:19,258 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [ ]:
# Generate masked Jacobian images for all local groups
GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [x[0:-1].split('\t')[1] for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(N_sub, sub_list, age_list)
    DATA_DIR='/data/'+current_group
    TMPT_list=['from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5', \
               'space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'];
    print('Processing ',current_group,' group start:')
    for i_sub in range(0, N_sub,1):
        import time
        t0=time.time()
        sub_str=sub_list[i_sub]
        print('Processing ',sub_str,' ( ',i_sub , ' ) start:')
        res1 = dataGraber_sub(sub_str, DATA_DIR, TMPT_list)
        H5_FILE=res1[0]
        WARP_OUT_DIR1=current_OUT_DIR+sub_str+'_'+'desc-preproc_T1w_space-MNI2009c'
        [out_warp, out_affine]=h5toWarp_nii(H5_FILE, WARP_OUT_DIR1)
        IN_IMAGE  = out_warp
        JACOB_IMG = current_OUT_DIR+sub_str+'_'+'desc-preproc_T1w_space-MNI2009c_Warp_Jacobian.nii.gz'
        JACOB_MASK_IMG = current_OUT_DIR+sub_str+'_'+'desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz'
        GetJacobian_nii(out_warp, res1[2], JACOB_IMG, JACOB_MASK_IMG)
        print('Processing ',sub_str,' finished in ',time.time()-t0, '\n')

40 ['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0129', 'sub-0132', 'sub-0136', 'sub-1000', 'sub-1020'] ['70', '76', '45', '63', '59', '57', '47', '66', '65', '53', '62', '50', '63', '44', '61', '72', '51', '70', '69', '68', '54', '63', '75', '54', '56', '66', '66', '48', '66', '73', '57', '69', '69', '54', '60', '69', '74', '62', '77', '6']
Processing  PD  group start:
Processing  sub-0002  (  0  ) start:
Grabbing files for:  sub-0002
dataGraber takes:  0.04472541809082031


In [1]:
# Read subject list, age list, and Jacobian Deter. of 3 study groups by sub_list 
import nipype.interfaces.io as nio
import os
# read data: 3 study groups by sub_list 
GROUPS=['PD','ET','NC']
OUT_DIR='/output/PD_ICA/'
SUB_LIST=[]; AGE_LIST=[]; JCOB_LIST=[];
print('Local data: ')
for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [int(x[0:-1].split('\t')[1]) for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(group_name, ': ', N_sub)
    # grab group Jacobians
    ds_jacobian = nio.DataGrabber(infields=['sub_id'])
    ds_jacobian.inputs.base_directory = current_OUT_DIR # database
    ds_jacobian.inputs.template = '%s_desc-preproc_T1w_space-MNI2009c_Warp_Jacobian-masked.nii.gz' # from cwd
    ds_jacobian.inputs.sort_filelist = True
    ds_jacobian.inputs.sub_id = sub_list
    res_jacobian = ds_jacobian.run()
    jacobian_list=res_jacobian.outputs.outfiles
    JCOB_LIST.append(jacobian_list)
pd_sub_list = SUB_LIST[0]; et_sub_list = SUB_LIST[1]; nc_sub_list = SUB_LIST[2]; 
pd_age_list = AGE_LIST[0]; et_age_list = AGE_LIST[1]; nc_age_list = AGE_LIST[2];
pd_jaco_list=JCOB_LIST[0]; et_jaco_list=JCOB_LIST[1]; nc_jaco_list=JCOB_LIST[2];

201003-18:15:17,312 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0
Local data: 
PD :  40
ET :  29
NC :  33


In [ ]:
# read AD ICAs 30 
import nipype.interfaces.io as nio
import os
PD_ICA_file='/templateflow/PD_ICAs';
ICs_list=list(range(30))
ICs_list=["{:0>4d}".format(x) for x in ICs_list]
# file import
ds_ic = nio.DataGrabber(infields=['IC_id'])
ds_ic.inputs.base_directory = PD_ICA_file # database
ds_ic.inputs.template = 'melodic_IC_%4s.nii.gz' # from cwd
ds_ic.inputs.sort_filelist = True
ds_ic.inputs.IC_id = ICs_list
res_ic = ds_ic.run()
ic_list=res_ic.outputs.outfiles

In [ ]:
# read local ICAs 19 
PD_ICA_local_file='/output/PD_ICA/ICA_local/stats';
ICs_local_list=[x+1 for x in list(range(19))]
#ICs_local_list=["{:0>4d}".format(x) for x in ICs_local_list]
# file import
ds_local_ic = nio.DataGrabber(infields=['IC_id'])
ds_local_ic.inputs.base_directory = PD_ICA_local_file # database
ds_local_ic.inputs.template = 'thresh_zstat%s.nii.gz' # from cwd
ds_local_ic.inputs.sort_filelist = True
ds_local_ic.inputs.IC_id = ICs_local_list
res_local_ic = ds_local_ic.run()
ic_local_list=res_local_ic.outputs.outfiles

In [4]:
# MELODIC ICA (local pd+nc Jacobian masked)
import time
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
ICA_LOCAL_PATH ='/output/PD_ICA/ICA_local'
if not os.path.exists(ICA_LOCAL_PATH):
    os.makedirs(ICA_LOCAL_PATH)
dir_all_jacobian = pd_jaco_list+nc_jaco_list
merged_file = '/output/PD_ICA/local_pd+nc_4d.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
melodic_setup.inputs.no_bet  = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
melodic_setup.inputs.out_pca = True
melodic_setup.inputs.out_dir = ICA_LOCAL_PATH
melodic_setup.inputs.report  = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  212.10636353492737
	MELODIC ICA takes:  2509.734400510788


In [ ]:
# plot and sv 30 AD ICs
from nilearn import plotting
GROUPS=['PD','ET','NC']
OUT_DIR='/output/PD_ICA/pics'
atlas_09_masked='/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz'
N_ic = len(ic_list)
i_ic=0
#for i_ic in range(N_ic):
#plotting.plot_anat(ic_list[i_ic], title="IC "+str(i_ic), )
VMIN= 3; VMAX=16; # vmin=VMIN;
#plotting.plot_glass_brain(ic_list[i_ic], title='no th '+str(i_ic)+', with TH=0', display_mode='lyrz',
#                          black_bg=True, colorbar=True, plot_abs=False, vmin=VMIN, vmax=VMAX, threshold=5)
for i_ic in range(N_ic):
    plotting.plot_stat_map(ic_list[i_ic], bg_img=atlas_09_masked, display_mode='z', threshold=3, vmax=VMAX, \
                           title="IC "+str(i_ic), draw_cross=False, cut_coords=[-16, -12, -7, -2,  8, 14, 20, 70], \
                           output_file=OUT_DIR+'/IC-'+str(i_ic)+'.png')

In [4]:
# MELODIC ICA (local et+nc Jacobian masked)
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
ICA_LOCAL_PATH ='/output/PD_ICA/ICA_local_et+nc'
if not os.path.exists(ICA_LOCAL_PATH):
    os.makedirs(ICA_LOCAL_PATH)
dir_all_jacobian = et_jaco_list+nc_jaco_list
merged_file = '/output/PD_ICA/local_et+nc_4d.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
melodic_setup.inputs.no_bet = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.out_pca = True
melodic_setup.inputs.out_dir = ICA_LOCAL_PATH
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA (ET+NC) takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  659.0801422595978
	MELODIC ICA (ET+NC) takes:  25265.937281131744


In [5]:
# MELODIC ICA (local pd+et+nc Jacobian masked)
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
ICA_LOCAL_PATH ='/output/PD_ICA/ICA_local_pd+et+nc'
if not os.path.exists(ICA_LOCAL_PATH):
    os.makedirs(ICA_LOCAL_PATH)
dir_all_jacobian = pd_jaco_list+et_jaco_list+nc_jaco_list
merged_file = '/output/PD_ICA/local_pd+et+nc_4d.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
melodic_setup.inputs.no_bet = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.out_pca = True
melodic_setup.inputs.out_dir = ICA_LOCAL_PATH
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA (PD+ET+NC) takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  235.3247435092926
	MELODIC ICA (PD+ET+NC) takes:  4546.186153411865
